# **Naver Book Search Open API**
### **목표**
1. **네이버 도서 검색 API 호출**:  
   - 키워드를 기반으로 도서 데이터를 검색합니다.
2. **데이터 분석 및 활용**:  
   - 검색된 데이터를 정리 및 가공하여 원하는 정보를 추출합니다.

### **작성자 정보**
Written by [**Yujin_nKim**](https://github.com/Yujin-nKim)

In [3]:
import secret
# 네이버 API 인증 키 설정 
# 네이버 개발자 센터에서 발급받은 클라이언트 ID와 시크릿 키를 이용합니다. 
NAVER_CLIENT_ID = secret.my_client_id # 네이버 클라이언트 ID
NAVER_CLIENT_SECRET = secret.my_client_secret # 네이버 클라이언트 시크릿 키 

# 네이버 도서 검색 API의 엔드포인트 URL (json 형식)
BOOK_SEARCH_JSON_URL = "https://openapi.naver.com/v1/search/book.json";
# 네이버 도서 상세 검색 API의 엔드포인트 URL (xml 형식)
BOOK_ADVANCED_SEARCH_XML_URL = "https://openapi.naver.com/v1/search/book_adv.xml";

In [4]:
# 검색 파라미터 설정
searchWord = "재테크" # 검색어
display = 100 # 한 번에 표시할 검색 결과 개수 (기본값: 10, 최댓값: 100)
start = 1 # 검색 시작 위치 (기본값: 1, 최댓값: 1000)
sort = "sim" # 검색 결과 정렬 방식 (sim - 정확도순 (기본값, 내림차순) / date - 날짜순 (내림차순)))

In [5]:
# API 호출 결과로 받은 도서 데이터 리스트를 출력하는 함수 
# items (list) : 도서 검색 결과의 아이템 목록
def print_item(items):
    for item in items:
        print("제목:", item["title"].replace("<b>", "").replace("</b>", ""))    
        print("링크:", item["link"])
        print("이미지링크:", item["image"])
        print("저자:", item["author"])
        print("가격:", item["discount"])
        print("출판사:", item["publisher"])
        print("출판일:", item["pubdate"])
        print("isbn:", item["isbn"])
        print("요약:", item["description"].replace("<b>", "").replace("</b>", ""))
        print("-" * 50)

In [6]:
import csv

# 도서 검색 결과를 CSV 파일로 저장하는 함수
# items (list) : 도서 검색 결과의 아이템 목록
# filename (str) : 저장할 CSV 파일 이름 (기본값: 'books_results.csv')
def save_to_csv(items, filename="books_results.csv"):
    # CSV 파일 열기
    with open(filename, mode="w", encoding="utf-8", newline="") as file:
        writer = csv.writer(file)
        
        # 헤더 작성
        writer.writerow(["제목", "링크", "이미지 링크", "저자", "가격", "출판사", "출판일", "isbn", "요약"])
        
        # 데이터 작성
        for item in items:
            title = item["title"].replace("<b>", "").replace("</b>", "")
            link = item["link"]
            image = item["image"]
            author = item["author"]
            discount = item["discount"]
            publisher = item["publisher"]
            pubdate = item["pubdate"]
            isbn = item["isbn"]
            description = item["description"].replace("<b>", "").replace("</b>", "")
            
            writer.writerow([title, link, image, author, discount, publisher, pubdate, isbn, description])

    print(f"결과가 {filename}에 저장되었습니다.")

In [7]:
# 네이버 검색 API 예제 - 도서 검색

import urllib.request
import json

# 네이버 API 인증 정보
client_id = NAVER_CLIENT_ID 
client_secret = NAVER_CLIENT_SECRET

all_items = [] # 모든 뉴스 데이터를 저장할 리스트

while True:
    # 검색 파라미터 설정
    query_params = {
        "query": urllib.parse.quote(searchWord),
        "display": display, 
        "start": start,
        "sort": sort
    }

    query_string = "&".join([f"{key}={value}" for key, value in query_params.items()])
    url = f"{BOOK_SEARCH_JSON_URL}?{query_string}"

    # API 요청 URL 출력
    print("요청 URL:", url)

    # 요청 객체 생성
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    # API 요청 및 응답 처리 
    try:
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if(rescode==200): # 성공 응답 
            response_body = response.read().decode('utf-8')
            data = json.loads(response_body)
            
            # 전체 결과 수와 현재 페이지의 아이템 가져오기
            #total_results = data["total"]
            total_results = 200 # test
            items = data["items"]
            all_items.extend(items)
            
            print(f"{start}번째 결과부터 {start + len(items) - 1}까지 데이터를 가져왔습니다.")
            
            # 다음 페이지로 이동
            start += display
            
            # 더 이상 가져올 데이터가 없으면 중단
            if start > total_results or not items:
                break
        else:
            print(f"Error Code: {response.getcode()}")
            break
    except Exception as e:
        print("Exception:", e)
        
# 모든 데이터 CSV로 저장
save_to_csv(all_items)

요청 URL: https://openapi.naver.com/v1/search/book.json?query=%EC%9E%AC%ED%85%8C%ED%81%AC&display=100&start=1&sort=sim
1번째 결과부터 100까지 데이터를 가져왔습니다.
요청 URL: https://openapi.naver.com/v1/search/book.json?query=%EC%9E%AC%ED%85%8C%ED%81%AC&display=100&start=101&sort=sim
101번째 결과부터 200까지 데이터를 가져왔습니다.
결과가 books_results.csv에 저장되었습니다.
